In [2]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv("data/travaux-engages-metropole-lyon.csv", sep=";")
print(df.describe())
display(df)

### Pretraitement

In [15]:
# --- 1. GÉNÉRATION DU FICHIER POINTS ROUGES (Coordonnées + Description) ---

with open('data_coord/points-rouges-200046977.geojson', 'r') as f:
    geojson_data = json.load(f)

# Extraction des coordonnées (longitude, latitude) et des propriétés
points_rouges_data = []
for feature in geojson_data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']
    
    points_rouges_data.append({
        'longitude': coords[0],
        'latitude': coords[1],
        'commune_insee': props.get('commune'), # 
        'description': props.get('description') # 
    })

df_points_rouges_final = pd.DataFrame(points_rouges_data)

# Sauvegarde du premier fichier
df_points_rouges_final.to_csv('points_rouges_lyon_complet.csv', index=False)


# --- 2. GÉNÉRATION DU FICHIER RÉPONSES (Questionnaire + Lieu) ---

# Chargement du CSV original
df_reponses = pd.read_csv('data_coord/reponses-epci-200046977.csv', low_memory=False, quotechar='"')

# Sélection des colonnes stratégiques selon la notice [cite: 86, 88, 90]
cols_reponses = [
    'insee',        # Lieu (Code INSEE) [cite: 21, 29]
    'q6',           # Fréquence de pratique [cite: 86]
    'q14',          # Sentiment de sécurité [cite: 86]
    'q18',          # Dangerosité carrefours [cite: 88]
    'q21',          # État de l'entretien [cite: 88]
    'q28',          # Conflit stationnement motorisé [cite: 88]
    'q34_texte',    # Les 3 priorités d'amélioration [cite: 88]
    'q35',          # Commentaire libre (NLP) [cite: 88]
    'q38_texte',    # Situations de violence vécues [cite: 90]
    'score'         # Note globale de la commune [cite: 30]
]

# Filtrage et nettoyage
df_reponses_final = df_reponses[cols_reponses].copy()
df_reponses_final['q35'] = df_reponses_final['q35'].str.replace(r'\r+|\n+', ' ', regex=True).fillna('')

# Sauvegarde du second fichier
df_reponses_final.to_csv('reponses_questionnaire_lyon.csv', index=False)

print("Les deux fichiers ont été générés avec succès :")
print("- points_rouges_lyon_complet.csv (Données cartographiques)")
print("- reponses_questionnaire_lyon.csv (Données thématiques et profils)")

Les deux fichiers ont été générés avec succès :
- points_rouges_lyon_complet.csv (Données cartographiques)
- reponses_questionnaire_lyon.csv (Données thématiques et profils)


### NLP

In [21]:
import pandas as pd
import numpy as np
from collections import Counter
import re

# --- 1. CONFIGURATION RÉFÉRENTIELLE ---
# On centralise ici la connaissance métier : Urgence et Facilité par type de problème
CONFIG_PROBLEMES = {
    'Infrastructure manquante': {'mots': r'piste|manque|absence|discontinuité|coupure', 'urgence': 3, 'facilite': 3},
    'Carrefours dangereux': {'mots': r'carrefour|intersection|rond-point|traversée', 'urgence': 3, 'facilite': 3},
    'Vitesse excessive': {'mots': r'vitesse|rapide|trop vite|ralentir', 'urgence': 3, 'facilite': 2},
    'Violence routière': {'mots': r'violence|insulte|intimidation|agression', 'urgence': 3, 'facilite': 2},
    'Entretien pistes': {'mots': r'entretien|entretenir|nid de poule|trou|verre', 'urgence': 2, 'facilite': 1},
    'Stationnement gênant': {'mots': r'stationnement|garé|voiture|parking|double file', 'urgence': 2, 'facilite': 2},
    'Stationnement vélo': {'mots': r'stationnement vélo|parking vélo|garage vélo', 'urgence': 2, 'facilite': 1},
    'Limitation trafic': {'mots': r'limiter trafic|réduire trafic|moins de voiture', 'urgence': 2, 'facilite': 3},
    'Signalisation': {'mots': r'panneau|feu|signalisation|marquage', 'urgence': 1, 'facilite': 1},
    'Conflit piétons': {'mots': r'piéton|trottoir|partagé|quai', 'urgence': 1, 'facilite': 2},
}

def get_label_facilite(score):
    if score <= 1.5: return "🟢 Facile"
    if score <= 2.5: return "🟡 Moyen"
    return "🔴 Difficile"

# --- 2. FONCTIONS D'ANALYSE ---
def detecter_themes(texte):
    if pd.isna(texte) or texte == "": return []
    return [nom for nom, cfg in CONFIG_PROBLEMES.items() if re.search(cfg['mots'], str(texte).lower())]

# --- 3. CHARGEMENT ET TRAITEMENT ---
print("🚀 Chargement des données...")
df = pd.read_csv('reponses_questionnaire_lyon.csv', low_memory=False)

# Fusion des textes et détection
df['verbatim'] = df[['q34_texte', 'q35', 'q38_texte']].fillna('').agg(' '.join, axis=1)
df['problemes_list'] = df['verbatim'].apply(detecter_themes)

# --- 4. ANALYSE PAR TYPE DE PROBLÈME (FACILITÉ GÉNÉRALE) ---
print("\n📊 ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME")
all_p = [p for sub in df['problemes_list'] for p in sub]
counts_global = Counter(all_p)

analyse_p = []
for nom, cfg in CONFIG_PROBLEMES.items():
    analyse_p.append({
        'Problème': nom,
        'Citations': counts_global[nom],
        'Urgence': cfg['urgence'],
        'Facilité': cfg['facilite'],
        'Type': get_label_facilite(cfg['facilite'])
    })

df_facilite_theme = pd.DataFrame(analyse_p).sort_values(by='Facilité')
print(df_facilite_theme.to_string(index=False))

# --- 5. ANALYSE PAR COMMUNE (TOP 10 + FACILITÉ MOYENNE) ---
print("\n ANALYSE PAR COMMUNE (TOP 10 ET FAISABILITÉ)")

resultats_communes = []
for insee, group in df.groupby('insee'):
    # Extraction de tous les problèmes de la commune
    commune_probs = [p for sub in group['problemes_list'] for p in sub]
    top_10_list = Counter(commune_probs).most_common(10)
    
    if top_10_list:
        # Calcul de la facilité moyenne des problèmes cités dans cette commune
        moy_facilite = np.mean([CONFIG_PROBLEMES[p]['facilite'] for p, _ in top_10_list])
        top_10_str = " | ".join([f"{p} ({c})" for p, c in top_10_list])
        
        resultats_communes.append({
            'INSEE': insee,
            'Réponses': len(group),
            'Sécurité_Moy': round(group['q14'].mean(), 2),
            'Facilité_Action': get_label_facilite(moy_facilite),
            'Top_10_Problemes': top_10_str
        })

df_communes = pd.DataFrame(resultats_communes).sort_values(by='Réponses', ascending=False)

# --- 6. AFFICHAGE FINAL ---
pd.set_option('display.max_colwidth', 100)
print("\n" + "="*120)
print(df_communes.head(15).to_string(index=False))
print("="*120)



🚀 Chargement des données...

📊 ANALYSE DE LA FACILITÉ PAR TYPE DE PROBLÈME
                Problème  Citations  Urgence  Facilité        Type
        Entretien pistes       5754        2         1    🟢 Facile
      Stationnement vélo         32        2         1    🟢 Facile
           Signalisation        465        1         1    🟢 Facile
       Vitesse excessive       9720        3         2     🟡 Moyen
       Violence routière       7171        3         2     🟡 Moyen
    Stationnement gênant       5199        2         2     🟡 Moyen
         Conflit piétons       6363        1         2     🟡 Moyen
Infrastructure manquante      10169        3         3 🔴 Difficile
    Carrefours dangereux        348        3         3 🔴 Difficile
       Limitation trafic         11        2         3 🔴 Difficile

 ANALYSE PAR COMMUNE (TOP 10 ET FAISABILITÉ)

 INSEE  Réponses  Sécurité_Moy Facilité_Action                                                                                               